In [99]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import warnings
warnings.simplefilter(action='ignore')

In [96]:
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [97]:
class neuroverkko(nn.Module):
    def __init__(self):
        super(neuroverkko, self).__init__()
        self.hidden = nn.Linear(4, 10)  # Piilotettu kerros, jossa 10 neuronia
        self.output = nn.Linear(10, 3)  # Ulostulokerros, jossa 3 neuronia

    def forward(self, x):
        x = torch.relu(self.hidden(x))  # Aktivointifunktio piilotetussa kerroksessa
        x = self.output(x)  # Ulostulo ilman softmaxia (CrossEntropyLoss hoitaa tämän)
        return x

model = neuroverkko()

In [98]:
# Koulutussilmukka
for epoch in range(50):  # Kouluta 50 epookkia
    model.train()  # Laita malli koulutustilaan
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()  # Nollaa vanhat gradit
        outputs = model(batch_X)  # Syötä yksi batch dataa mallille
        loss = criterion(outputs, batch_y)  # Laske tappio tälle batchille
        loss.backward()  # Laske gradit
        optimizer.step()  # Päivitä painot

    # Lasketaan tarkkuus validointidatalla
    model.eval()  # Laita malli arviointitilaan
    correct = 0
    total = 0
    with torch.no_grad():  # Ei lasketa gradientteja validoinnissa
        for batch_X, batch_y in val_loader:
            val_outputs = model(batch_X)
            _, predicted = torch.max(val_outputs, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}, Accuracy: {accuracy:.2f}%")


Epoch 1, Accuracy: 33.33%
Epoch 2, Accuracy: 33.33%
Epoch 3, Accuracy: 33.33%
Epoch 4, Accuracy: 33.33%
Epoch 5, Accuracy: 33.33%
Epoch 6, Accuracy: 33.33%
Epoch 7, Accuracy: 33.33%
Epoch 8, Accuracy: 33.33%
Epoch 9, Accuracy: 33.33%
Epoch 10, Accuracy: 33.33%
Epoch 11, Accuracy: 33.33%
Epoch 12, Accuracy: 33.33%
Epoch 13, Accuracy: 33.33%
Epoch 14, Accuracy: 33.33%
Epoch 15, Accuracy: 33.33%
Epoch 16, Accuracy: 33.33%
Epoch 17, Accuracy: 33.33%
Epoch 18, Accuracy: 33.33%
Epoch 19, Accuracy: 33.33%
Epoch 20, Accuracy: 33.33%
Epoch 21, Accuracy: 33.33%
Epoch 22, Accuracy: 33.33%
Epoch 23, Accuracy: 33.33%
Epoch 24, Accuracy: 33.33%
Epoch 25, Accuracy: 33.33%
Epoch 26, Accuracy: 33.33%
Epoch 27, Accuracy: 33.33%
Epoch 28, Accuracy: 33.33%
Epoch 29, Accuracy: 33.33%
Epoch 30, Accuracy: 33.33%
Epoch 31, Accuracy: 33.33%
Epoch 32, Accuracy: 33.33%
Epoch 33, Accuracy: 33.33%
Epoch 34, Accuracy: 33.33%
Epoch 35, Accuracy: 33.33%
Epoch 36, Accuracy: 33.33%
Epoch 37, Accuracy: 33.33%
Epoch 38, 